#### Nome: Matheus Maia da Silva
#### Matrícula: 19204136

# 1) Faça um crawler que navegue pelas páginas de países e baixe os htmls.

##### -- O algoritmo desta parte entra na primeira página de países, pega os links da páginas de todos os países e os coloca em um dicionário em que as chaves são os nomes do países. Depois disso, procurar pelo link da próxima página de países e repete o processo até a última página.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


# Faz a requisição do html da primeira página
response = requests.get('http://127.0.0.1:8000/places/default/index/')
soup = BeautifulSoup(response.content)
response.close()

# Irá guardar todos os links de referêcia para o botão 'next' (usado para parar o loop)
all_page_links = []

# Dicionário que irá guardar o link de todos os países
pages = {}


# Loop que irá navegar pelas páginas de países e irá extrair seus respectivos links
while True:
    # Cria uma lista com os nomes dos paises
    names = [name.text.strip() for name in soup.body.table.find_all('a')]

    # Cria uma lista com os links para a página dos países
    links = list(map(lambda x: x.get('href'), soup.body.section.find_all('a')))[0:-1]

    # Cria um dicionário com o nome dos países, um número de referência e seus links
    for idx, name in enumerate(names):
        pages.update({name: links[idx]})


    # Padrão a ser procurado
    pattern = '/places/default/index/'

    # Fução que procura um padrão de link
    def search_pattern(link):
        if pattern in link:
            return link
        

    # Sequência que filtra os links para encontrar apenas o links que contém o botão 'next' e 'previous'
    link_next = list(map(lambda x: x['href'], soup.body.section.find_all('a')))
    link_next = list(map(search_pattern, link_next))
    link_next = list(filter(None, link_next))


    # Para o loop quando o único link for o 'previous' (ou seja, quando um link repetido aparecer)
    if link_next[-1] in all_page_links:
        break
    else:
        all_page_links.append(link_next[-1])


    # Faz o requerimento e extrai o html da página próxima página
    soup = BeautifulSoup(requests.get('http://127.0.0.1:8000' + link_next[-1]).content)


print(f'Tamanho do dicionário de países: {len(pages)}')
display(pages)

# 2) Faça scraping dos htmls baixados e salve os dados retirados em um arquivo csv. Salvar uma coluna extra no csv contendo um timestamp do momento no qual os dados foram obtidos.

#### -- O bloco abaixo cria um dataframe o pandas onde os dados de todos os países serão armazenados para posteriormente criar o CSV.

In [ ]:
# Faz uma requisição do primeiro país para extrair as colunas do dataframe
response = requests.get('http://127.0.0.1:8000/places/default/view/Afghanistan-1')
soup = BeautifulSoup(response.content)
response.close()

# Extrai as informações do país que está no html
infos = [info.text.strip() for info in soup.body.table]


# Criação do dataframe usando pandas
columns = []
for i in infos:
    columns.append(i.split(':')[0].lower().replace(' ', '_'))
columns.append('timestamp')
df = pd.DataFrame(columns=columns)


print('Dataframe vazio: ')
display(df)

#### -- Abaixo são feita as requisições das páginas de todos os países, extraindo suas informações e preenchendo o dataframe. Após isso é criado o arquivo CSV.

In [ ]:
for country, link in pages.items():
    # Faz uma requisição da página do país
    response = requests.get('http://127.0.0.1:8000' + link)
    response.close()
    soup = BeautifulSoup(response.content)
    


    # Extrai os dados
    data = [tag.text.split(':')[-1].strip() for tag in soup.body.section.div.form.table.find_all('tr')]
    data.append(response.headers['Date'])


    # Faz a limpeza dos dados
    data[0] = None
    data[1] = int(data[1].split(' ')[0])
    data[2] = int(data[2])
    data[11] = data[12] = None
    data[13] = data[13].replace(',', ' ')

    def check_if_empty(value):
        if value == '':
            return None
        else:
            return value
        
    data = list(map(check_if_empty, data))


    # Acrescenta uma nova linha ao dataframe
    new_row = pd.DataFrame(dict(zip(columns, data)), index=[0])
    df = pd.concat([df, new_row], ignore_index=True)


# Converte o dataframe em CSV
df.to_csv('countries.csv', index=False)


response.close()
display(df.head())